## Ablation study - czym jest?
Zgodnie ze artykułem na wikipedii : https://en.wikipedia.org/wiki/Ablation_(artificial_intelligence)  
Ablation study jest zajmuje się badaniem wydajności systemów sztucznej inteligencji poprzez usuwanie niektórych części systemu by móc lepiej zrozumieć jego rolę, zaangażowanie w całość. Badanie zdegradowanych systemów do momentu całkowitego zaprzestania ich funkcjonalności, niesie ze sobą informację o wytrzymałości, wrażliwości oraz złożoności systemu.  
Tak jak wedle artykuły na wikipedii, pojęcie występuje w biologii i tyczy się odpowiednio równoważnego pojęcia badania zdeprawowanych z pewnych systemów organizmów i obserwacji ich funkcjonalności.

## Przygotowanie ramki danych oraz sieci neuronowej
W moim Ablation study operować będę na zbiorze CIFAR 10, oraz na autorsko stworzonej sieci neuronowej.  
Źródło do zbioru danych CIFAR-10 : https://www.cs.toronto.edu/~kriz/cifar.html  

In [2]:
## Załadujmy potrzebne pakiety
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import keras
from sklearn.preprocessing import OneHotEncoder

In [3]:
(X_train,y_train), (X_test,y_test) = datasets.cifar10.load_data()
print(X_train.shape)
print(X_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


Zanim przejdziemy dalej, muszę dorzucić kilka słów a propos naszej ramki danych. Jak wiemy, CIFAR-10 zawiera 60000 grafik rbg z 10 klasami, w postaci enumeracyjnej z wartościami:  
0 - airplane  
1 - automoblie  
2 - bird  
3 - cat  
4 - deer  
5 - dog  
6 - frog  
7 - horse  
8 - ship  
9 - truck  
Jednakże zerknijmy teraz na nasze y_train oraz y_test.

In [4]:
print(y_train[:5], "\n")
print(y_test[:5])

[[6]
 [9]
 [9]
 [4]
 [1]] 

[[3]
 [8]
 [8]
 [0]
 [6]]


Jak widzimy jest to dwuwymiarowy wektor, jednakże taki obiekt jest całkowicie zbędny, więc zastąpmy go po prostu jedno wymiarowym. Następnie będziemy mogli także dokonać łatwiej enkodowania zmiennych, np. onehot.

In [5]:
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

In [6]:
#Teraz lepiej
print(y_train[:5], "\n")
print(y_test[:5])

[6 9 9 4 1] 

[3 8 8 0 6]


Dla optymalizacji zeskalujmy teraz dane w zbiorach testowym oraz treningowym oraz zastosujmy enkodowanie dla zmiennych numerycznych w y_test oraz y_train.

In [7]:
X_test = X_test /255
X_train = X_train /255

In [21]:
X_train.shape

(50000, 32, 32, 3)

## Tworzenie modelu
Ok, początek mamy z głowy, teraz przejdźmy do ciekawszej części zadania, czyli do utworzenia modelu sieci neuronowej, w celach strzeszczeniowych dany model nie będzie przesadnie rozbudowany, lecz będzie spełniał wymogi zadania. Dla niektórych zmiennych w warstwach sieci neuronowej, ich wartość jest równa ich wartości domyślnej, oznacza to, iż będę eksperymentował z tymi danymi i należało to podkreślić w tworzeniu modelu.

In [8]:
original_model = keras.Sequential(
[
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

Ok, zanim przejdziemy do ablation study, sprawdźmy poprawność powyższego modelu.

In [83]:
print(original_model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 4096)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                262208    
_________________________________________________________________
dense_17 (Dense)             (None, 10)              

In [9]:
original_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

original_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 49s 31ms/step - loss: 1.6325 - accuracy: 0.4107
Epoch 2/8
1563/1563 [==============================] - 46s 29ms/step - loss: 1.0426 - accuracy: 0.6361
Epoch 3/8
1563/1563 [==============================] - 48s 30ms/step - loss: 0.9206 - accuracy: 0.6787
Epoch 4/8
1563/1563 [==============================] - 46s 30ms/step - loss: 0.8239 - accuracy: 0.7149
Epoch 5/8
1563/1563 [==============================] - 48s 31ms/step - loss: 0.7421 - accuracy: 0.7408
Epoch 6/8
1563/1563 [==============================] - 46s 30ms/step - loss: 0.6808 - accuracy: 0.7640
Epoch 7/8
1563/1563 [==============================] - 45s 29ms/step - loss: 0.6254 - accuracy: 0.7804
Epoch 8/8
1563/1563 [==============================] - 45s 29ms/step - loss: 0.5604 - accuracy: 0.8037


In [10]:
## Sprawdźmy jak sprawuje się model na danych testowych
original_model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 8ms/step - loss: 0.9398 - accuracy: 0.6895


[0.9398031234741211, 0.6894999742507935]

Uzyskalismy około 73% accuracy, co dla 4 epok jest nie najgorszym wynikiem. Ograniczam się wyłącznie do 4 epok, ze względu na czasochłonnosć mojego modelu oraz względnie dobrego wyniku accuracy.

## Właściwe albation study - dodawanie/usuwanie warstw
Przejdźmy wreszcie do właściwej części naszego zadania, czyli do eksperymentowania z naszymi parametrami. Na początku utwórzmy pewne inne, zmodyfikowane wersje naszej oryginalnej sieci neuronowej.  
Zacznijmy z usuwaniem warstw, gdyż, wedle mnie, łatwo pokaże nam to, które warstwy naszej sieci są najbardziej newralgiczne.

In [12]:
# W oryginalnym modelu nie uwzględniłem warstyw dropout chroniącej przed przeuczaniem, sprawdźmy czy dodanie ów, zmieni wynik
mod_1_model = keras.Sequential(
[  
    keras.Input(shape = (32,32,3)),
    layers.Dropout(0.15),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [13]:
mod_1_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_1_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 49s 31ms/step - loss: 1.7520 - accuracy: 0.3656
Epoch 2/8
1563/1563 [==============================] - 48s 31ms/step - loss: 1.2470 - accuracy: 0.5554
Epoch 3/8
1563/1563 [==============================] - 48s 31ms/step - loss: 1.1073 - accuracy: 0.6104
Epoch 4/8
1563/1563 [==============================] - 48s 31ms/step - loss: 1.0273 - accuracy: 0.6399
Epoch 5/8
1563/1563 [==============================] - 47s 30ms/step - loss: 0.9586 - accuracy: 0.6620
Epoch 6/8
1563/1563 [==============================] - 47s 30ms/step - loss: 0.9128 - accuracy: 0.67790s - loss: 0.9128 - accuracy: 0.
Epoch 7/8
1563/1563 [==============================] - 47s 30ms/step - loss: 0.8760 - accuracy: 0.6902
Epoch 8/8
1563/1563 [==============================] - 49s 32ms/step - loss: 0.8356 - accuracy: 0.7076


In [14]:
#Podobny wynik dla zbioru testowego
mod_1_model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 8ms/step - loss: 1.3924 - accuracy: 0.5328


[1.3924410343170166, 0.532800018787384]

Zauważyliśmy pwien spadek accuracy, zatem nasz model potrzebuje większej ilości danych by nauczyć się lepiej, oraz wyłączenie  15% neuronów przy wejściu obniżył jakość modelu o 10 punktów procentowych, również wejście powłoki Dropout usuytuowane na początku powinna "na logikę" zmieniać wagai pomiędzy neuronami w największym stopniu. O ironio, zabieg, który w zamiarze miał przeciwdziałać przeuczeniu, na zbiorze testowym wykazał się gorszą accuracy :(.    
Spróbujmy jeszcze raz z warstwą dropout, tym razem przy samym końcu naszych warstw, czy nasza "logika" sprawdzi się i accuracy będzie większe?.

In [15]:
mod_2_model = keras.Sequential(
[  
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dropout(0.15),
    layers.Dense(10,activation = 'softmax'),
]
)

In [16]:
mod_2_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_2_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 53s 34ms/step - loss: 1.7202 - accuracy: 0.3719
Epoch 2/8
1563/1563 [==============================] - 51s 33ms/step - loss: 1.1587 - accuracy: 0.5905
Epoch 3/8
1563/1563 [==============================] - 50s 32ms/step - loss: 0.9953 - accuracy: 0.6488
Epoch 4/8
1563/1563 [==============================] - 50s 32ms/step - loss: 0.9259 - accuracy: 0.6728
Epoch 5/8
1563/1563 [==============================] - 50s 32ms/step - loss: 0.8608 - accuracy: 0.6949
Epoch 6/8
1563/1563 [==============================] - 51s 33ms/step - loss: 0.8005 - accuracy: 0.71850s - loss: 0.8005 - accu
Epoch 7/8
1563/1563 [==============================] - 52s 33ms/step - loss: 0.7388 - accuracy: 0.7392
Epoch 8/8
1563/1563 [==============================] - 51s 32ms/step - loss: 0.7074 - accuracy: 0.7471


In [17]:
#Podobny wynik dla zbioru testowego
mod_2_model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 9ms/step - loss: 1.0071 - accuracy: 0.6700


[1.0070972442626953, 0.6700000166893005]

Ok, jest lepiej niż poprzednio, co zakładałem uprzednio. Zanim przejdziemy do innego aspektu naszych powłok, dokonajmy testu dla warstwy dropout **przed** warstwą Flatten.

In [18]:
mod_3_model = keras.Sequential(
[  
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    layers.Dropout(0.15),
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [19]:
mod_3_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_3_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 53s 34ms/step - loss: 1.6673 - accuracy: 0.3961
Epoch 2/8
1563/1563 [==============================] - 58s 37ms/step - loss: 1.1082 - accuracy: 0.6127
Epoch 3/8
1563/1563 [==============================] - 59s 38ms/step - loss: 0.9536 - accuracy: 0.6678
Epoch 4/8
1563/1563 [==============================] - 54s 35ms/step - loss: 0.8753 - accuracy: 0.6981
Epoch 5/8
1563/1563 [==============================] - 56s 36ms/step - loss: 0.7952 - accuracy: 0.7224
Epoch 6/8
1563/1563 [==============================] - 58s 37ms/step - loss: 0.7394 - accuracy: 0.7418
Epoch 7/8
1563/1563 [==============================] - 55s 35ms/step - loss: 0.6972 - accuracy: 0.7561
Epoch 8/8
1563/1563 [==============================] - 57s 37ms/step - loss: 0.6553 - accuracy: 0.7718


In [20]:
#Podobny wynik dla zbioru testowego
mod_3_model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.8906 - accuracy: 0.7001


[0.8906192183494568, 0.7001000046730042]

Efekt jest praktcznie identyczny, choć lepszy, również accuracy dla zbiory testowego jest również lepsze, czyli dropout spełnił swoje zadanie :D.

No dobrze dodawaliśmy, teraz czas zabrać. Stosowaliśmy dwie warstwy konwolucyjne oraz dwie warstwy MaxPoolingu, jednakże jak wiemy z opisu zbioru danych CIFAR-10, zamieszczone tam grafiki są nieznacznych rozdzielczości, więc zastosowanie podwójnej konwolucji może okazać się zbyt dużym przedsięwzięciem. Zatem zabierzmy jedną z nich, wraz z MaxPooling.

In [21]:
mod_4_model = keras.Sequential(
[  
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    layers.Flatten(),
    layers.Dense(32,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [22]:
mod_4_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_4_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 26s 17ms/step - loss: 1.7901 - accuracy: 0.3578
Epoch 2/8
1563/1563 [==============================] - 23s 15ms/step - loss: 1.2856 - accuracy: 0.5475
Epoch 3/8
1563/1563 [==============================] - 22s 14ms/step - loss: 1.1591 - accuracy: 0.5916
Epoch 4/8
1563/1563 [==============================] - 22s 14ms/step - loss: 1.0793 - accuracy: 0.6160
Epoch 5/8
1563/1563 [==============================] - 24s 16ms/step - loss: 1.0259 - accuracy: 0.6387
Epoch 6/8
1563/1563 [==============================] - 22s 14ms/step - loss: 0.9748 - accuracy: 0.65650s - l
Epoch 7/8
1563/1563 [==============================] - 28s 18ms/step - loss: 0.9409 - accuracy: 0.66650s - loss: 0.9408 - accuracy: 
Epoch 8/8
1563/1563 [==============================] - 24s 15ms/step - loss: 0.9161 - accuracy: 0.6746


In [23]:
mod_4_model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1131 - accuracy: 0.6169


[1.1130871772766113, 0.6169000267982483]

Zabranie jednej warstwy konwolucyjnej wrawz z MaxPoolingiem zmniejszyło, choć znieznacznie, accuracy, jednakże wnioskować możemy, iż zwiększanie ilości warst konwolucyjnych nie będzie poprawiało wyników w nieskończnoność, w szczególności dla niewielkich grafik jak w naszym przypadku.  
Miałem w planach przeprowadzenie testu modelu z jeszcze jedną dodatkową warstwą konwolucyjną, lecz jakość mojego komputera nie pozwoliłaby mi na to. 

## Parametry w warstwach
W następnym kroku zabawmy się parametrami w istniejących już warstwach.  
Dla przypomnienia, nasz model prezentuje się następująco:

In [34]:
original_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 4096)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 64)                262208    
_________________________________________________________________
dense_21 (Dense)             (None, 10)              

Zacznijmy pierw od modyfikacji funkcji aktywującej dla wsztskich warstw, za wyjątkiem ostatniej. Relu jest dość popularną oraz sprawdzoną metodą jeżeli chodzi o funkcje aktywacji. Zatem przetestujmy coś bardziej "egzotycznego". Zobaczmy co się stanie dla funkcji exponential.

In [24]:
mod_5_model = keras.Sequential(
[
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'exponential'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'exponential'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    
    layers.Flatten(),
    layers.Dense(64,activation = 'exponential'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [25]:
mod_5_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_5_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 53s 33ms/step - loss: 752.5754 - accuracy: 0.0984
Epoch 2/8
1563/1563 [==============================] - 51s 32ms/step - loss: 2.3029 - accuracy: 0.0962
Epoch 3/8
1563/1563 [==============================] - 51s 32ms/step - loss: 2.3028 - accuracy: 0.1000
Epoch 4/8
1563/1563 [==============================] - 51s 33ms/step - loss: 2.3028 - accuracy: 0.1024
Epoch 5/8
1563/1563 [==============================] - 50s 32ms/step - loss: 2.3189 - accuracy: 0.1022
Epoch 6/8
1563/1563 [==============================] - 51s 33ms/step - loss: 2.3168 - accuracy: 0.0960
Epoch 7/8
1563/1563 [==============================] - 54s 34ms/step - loss: 2.3008 - accuracy: 0.1048
Epoch 8/8
1563/1563 [==============================] - 53s 34ms/step - loss: 2.2298 - accuracy: 0.1511


In [26]:
mod_5_model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 10ms/step - loss: 2.2894 - accuracy: 0.1028


[2.289391279220581, 0.10279999673366547]

Jest, celny strzał zatapiający naszą siec neuronową, accuracy naszej sieci zostało zdegradowane do ledwie 15%, lecz cóż od tego jest **Ablation study**.  No dobrze, spróbujmy jeszcze raz z funkcją aktywacji, tym razem z funkcją sigmoid, czym bardziej sprawdzonym.

In [27]:
mod_6_model = keras.Sequential(
[
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'sigmoid'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'sigmoid'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [28]:
mod_6_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_6_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 50s 32ms/step - loss: 2.3166 - accuracy: 0.0968
Epoch 2/8
1563/1563 [==============================] - 57s 36ms/step - loss: 2.3027 - accuracy: 0.0981
Epoch 3/8
1563/1563 [==============================] - 50s 32ms/step - loss: 2.3027 - accuracy: 0.0968
Epoch 4/8
1563/1563 [==============================] - 48s 30ms/step - loss: 2.3027 - accuracy: 0.0982
Epoch 5/8
1563/1563 [==============================] - 48s 31ms/step - loss: 2.3027 - accuracy: 0.1020
Epoch 6/8
1563/1563 [==============================] - 49s 32ms/step - loss: 2.3027 - accuracy: 0.0983
Epoch 7/8
1563/1563 [==============================] - 53s 34ms/step - loss: 2.3027 - accuracy: 0.1021
Epoch 8/8
1563/1563 [==============================] - 62s 40ms/step - loss: 2.3028 - accuracy: 0.1003


In [29]:
mod_6_model.evaluate(X_test,y_test)

313/313 [==============================] - 3s 8ms/step - loss: 2.3027 - accuracy: 0.1000


[2.302654981613159, 0.10000000149011612]

I kolejny model zatopiony, ok może zmieńmy cel naszych parametrów, w tym przypadku jednak dodatkowo czas wykonywania był znacząco dłuższy, funkcja sigmoid zabierała dużo czasu na kompilacje. Wiemy, że nasze dane są obrazami o rozdzielczości 32x32. W oryginalnym modelu użyliśmy dość niewielkiej wielkości pool'u (2,2) oraz użyliśmy paddingu = 'same', zatem warstwa konwolucji nie zmieniała rozmiarów grafiki. Zmieńmy padding na domyślny 'valid' oraz zwiększmy rozmiar pool'u do (4,4).

In [30]:
mod_7_model = keras.Sequential(
[
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="valid",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(4, 4)),
    layers.Conv2D(64,kernel_size = (3,3),padding="valid",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(4, 4)),
  
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [31]:
mod_7_model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

mod_7_model.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 18s 11ms/step - loss: 1.9123 - accuracy: 0.2858
Epoch 2/8
1563/1563 [==============================] - 18s 11ms/step - loss: 1.4452 - accuracy: 0.4813
Epoch 3/8
1563/1563 [==============================] - 17s 11ms/step - loss: 1.3248 - accuracy: 0.5266
Epoch 4/8
1563/1563 [==============================] - 20s 13ms/step - loss: 1.2346 - accuracy: 0.5642
Epoch 5/8
1563/1563 [==============================] - 18s 12ms/step - loss: 1.1733 - accuracy: 0.5870
Epoch 6/8
1563/1563 [==============================] - 18s 12ms/step - loss: 1.1318 - accuracy: 0.6007
Epoch 7/8
1563/1563 [==============================] - 18s 11ms/step - loss: 1.0905 - accuracy: 0.6159
Epoch 8/8
1563/1563 [==============================] - 18s 12ms/step - loss: 1.0610 - accuracy: 0.6280


In [32]:
mod_7_model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 5ms/step - loss: 1.1421 - accuracy: 0.6038


[1.1420811414718628, 0.6037999987602234]

Accuracy odrobinę niższe, lecz ostatecznie wyniki nie były najgorsze, niemniej jednak precyzyjniejsza analiza okazała się lepsza nawet dla niewielkich grafik, choć przy większym poolingu jest znacznie szybsza!

## Parametry treningu
Po uprzednich zniszczeniach spowodowanych naruszaniem homeostazy idealnych warunków modelu, teraz przyjrzyjmy się samemu treningowi oraz potencjalnym zmianom dla nauki modelu.  
Zacznijmy od sprawdzenia innego optimiziea, mianowicie dość popularnego SGD.

In [33]:
original_model_prim = keras.Sequential(
[
    keras.Input(shape = (32,32,3)),
    layers.Conv2D(32,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64,kernel_size = (3,3),padding="same",strides=(1,1),activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
  
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(10,activation = 'softmax'),
]
)

In [34]:
original_model_prim.compile(optimizer = 'SGD', loss = 'poisson',metrics=['accuracy'])

original_model_prim.fit(X_train,y_train,epochs = 8)

Epoch 1/8
1563/1563 [==============================] - 49s 31ms/step - loss: 10.4820 - accuracy: 0.0966
Epoch 2/8
1563/1563 [==============================] - 47s 30ms/step - loss: 10.4790 - accuracy: 0.0944
Epoch 3/8
1563/1563 [==============================] - 46s 30ms/step - loss: 10.5130 - accuracy: 0.0968
Epoch 4/8
1563/1563 [==============================] - 46s 29ms/step - loss: 10.4438 - accuracy: 0.1034 0s - loss: 10.4436 - accura
Epoch 5/8
1563/1563 [==============================] - 47s 30ms/step - loss: 10.4781 - accuracy: 0.1003
Epoch 6/8
1563/1563 [==============================] - 48s 31ms/step - loss: 10.4473 - accuracy: 0.0990
Epoch 7/8
1563/1563 [==============================] - 47s 30ms/step - loss: 10.4729 - accuracy: 0.0978 0s - loss: 10.4732 - acc
Epoch 8/8
1563/1563 [==============================] - 48s 31ms/step - loss: 10.5092 - accuracy: 0.0980


In [36]:
original_model_prim.evaluate(X_test,y_test)

313/313 [==============================] - 3s 8ms/step - loss: 10.4617 - accuracy: 0.0980


[10.461690902709961, 0.09799999743700027]

Optimizer oraz loss function mają newralgiczne znaczenie przy efektywności sieci neuronowej.

## Wnioski

Jak zobaczyliśmy powyżej **Ablation** study pozwolił nam na całkowite zdeprawowanie modelu, ale jak i również na znalezienie kilku równie dobrze działających alternatyw, choć niestety nie było żadnej lepszej pod względem accuracy.  
Wartym wypunktowania jest fakt, iż nawet niewieka wartość w modelu jak funkcja aktywacji może zadecydować o całkowitej funkcjonalności lub niefunkcjonalności sieci neuronowej, co pokazuje jak złożone i wrażliwe na zmiany są.  
Oczywiście nie jestem w stanie przeprowadzić pewnego odpowiednika "strojenia parametrów" z dużą precyzją jak w problemach machine learningu, gdyż złożoność sieci neuronowych przekracza możliwości mojego (i nie tylko mojego) komputera. Przy ogromnej różnorodności parametrów i ich wartości, takie zadanie okazuje się niemożliwe dla pojedyńczej jednostki.  
Niemniej jednak ablation study, tak jak jej odpowiednik w biologii, pokazuje jak pewne parametry pojedyńczo wpływają na całość.